In [0]:
import pandas as pd

In [23]:
engage = pd.read_csv('takehome_user_engagement.csv')
engage.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [24]:
engage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
time_stamp    207917 non-null object
user_id       207917 non-null int64
visited       207917 non-null int64
dtypes: int64(2), object(1)
memory usage: 4.8+ MB


In [27]:
visits = engage["user_id"].value_counts()
adopted = visits[visits >= 3]
 
adopted

3623    606
906     600
1811    593
7590    590
8068    585
       ... 
6090      3
8464      3
5528      3
6900      3
7310      3
Name: user_id, Length: 2248, dtype: int64

In [35]:
users = pd.read_csv('takehome_users.csv',encoding ='ISO-8859-1')
users.head(10)

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0
5,6,2013-12-17 03:37:06,Cunha Eduardo,EduardoPereiraCunha@yahoo.com,GUEST_INVITE,1.387424e+09,0,0,197,11241.0
6,7,2012-12-16 13:24:32,Sewell Tyler,TylerSewell@jourrapide.com,SIGNUP,1.356010e+09,0,1,37,NaN
7,8,2013-07-31 05:34:02,Hamilton Danielle,DanielleHamilton@yahoo.com,PERSONAL_PROJECTS,NaN,1,1,74,NaN
8,9,2013-11-05 04:04:24,Amsel Paul,PaulAmsel@hotmail.com,PERSONAL_PROJECTS,NaN,0,0,302,NaN
9,10,2013-01-16 22:08:03,Santos Carla,CarlaFerreiraSantos@gustr.com,ORG_INVITE,1.401833e+09,1,1,318,4143.0


In [36]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
object_id                     12000 non-null int64
creation_time                 12000 non-null object
name                          12000 non-null object
email                         12000 non-null object
creation_source               12000 non-null object
last_session_creation_time    8823 non-null float64
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
org_id                        12000 non-null int64
invited_by_user_id            6417 non-null float64
dtypes: float64(2), int64(4), object(4)
memory usage: 937.6+ KB


In [37]:
for index, val in adopted.iteritems(): 
    users.loc[users['object_id'] == index, 'adopted_user'] = 1 #index is used to connect with users df 'object_id' column
    
#Check that transformation went through with confirmed adopted user
users.loc[users['object_id'] == 3623]

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted_user
3622,3623,2012-06-25 14:34:33,Melo Vitória,VitoriaAlvesMelo@jourrapide.com,ORG_INVITE,1.401201e+09,0,0,364,3623.0,1.0


In [38]:
users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted_user
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,NaN
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,1.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,NaN
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,NaN
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,NaN


In [0]:
users = users.fillna(0)

In [41]:
users.drop(['creation_time', 'name', 'email', 'creation_source', 'last_session_creation_time', 'org_id', 'invited_by_user_id'], axis=1, inplace=True)
users

,object_id,opted_in_to_mailing_list,enabled_for_marketing_drip,adopted_user
0,1,1,0,0.0
1,2,0,0,1.0
2,3,0,0,0.0
3,4,0,0,0.0
4,5,0,0,0.0
...,...,...,...,...
11995,11996,0,0,0.0
11996,11997,0,0,0.0
11997,11998,1,1,0.0
11998,11999,0,0,0.0


In [42]:
train = users[:9600] #Training Set
train

,object_id,opted_in_to_mailing_list,enabled_for_marketing_drip,adopted_user
0,1,1,0,0.0
1,2,0,0,1.0
2,3,0,0,0.0
3,4,0,0,0.0
4,5,0,0,0.0
...,...,...,...,...
9595,9596,0,0,0.0
9596,9597,0,0,1.0
9597,9598,0,0,0.0
9598,9599,0,0,0.0


In [43]:
test = users[9600:] #Test Set (based off of Test set being 20% of whole dataset)
test

,object_id,opted_in_to_mailing_list,enabled_for_marketing_drip,adopted_user
9600,9601,1,0,1.0
9601,9602,0,0,0.0
9602,9603,0,0,0.0
9603,9604,0,0,0.0
9604,9605,0,0,0.0
...,...,...,...,...
11995,11996,0,0,0.0
11996,11997,0,0,0.0
11997,11998,1,1,0.0
11998,11999,0,0,0.0


In [0]:
trainx = train.drop('adopted_user', axis = 1)

trainy = train['adopted_user']

In [0]:
testx = test.drop('adopted_user', axis = 1)

testy = test['adopted_user']

In [0]:
from sklearn.linear_model import LogisticRegression

lgr = LogisticRegression(class_weight = 'balanced').fit(trainx, trainy)


In [48]:
lgr_train_pred =  lgr.predict(trainx)
lgr_train_pred

array([1., 1., 1., ..., 1., 1., 1.])

In [49]:
from sklearn.metrics import classification_report

report = classification_report(trainy, lgr_train_pred)
print(report)

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      7819
         1.0       0.19      1.00      0.31      1781

    accuracy                           0.19      9600
   macro avg       0.09      0.50      0.16      9600
weighted avg       0.03      0.19      0.06      9600



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [50]:
lgr_test_pred = lgr.predict(testx)
lgr_test_pred

array([1., 1., 1., ..., 1., 1., 1.])

In [51]:
from sklearn.metrics import classification_report

report = classification_report(testy, lgr_test_pred)
print(report)

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      1933
         1.0       0.19      1.00      0.33       467

    accuracy                           0.19      2400
   macro avg       0.10      0.50      0.16      2400
weighted avg       0.04      0.19      0.06      2400



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=33)

In [0]:
from sklearn.model_selection import GridSearchCV
param_grid = [
    { 'n_estimators': [3,10,30], 'max_features': [2,4,6,8]}
]
grid_search = GridSearchCV(rfc, param_grid, cv=5)

In [59]:
grid_search.fit(trainx,trainy)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: max_features must be in (0, n_features]

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: max_features must be in (0, n_features]

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: max_features must be in (0, n_features]

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this t

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False, random_state=33,
                                  

In [60]:
grid_search.best_estimator_

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=2,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=30,
                       n_jobs=None, oob_score=False, random_state=33, verbose=0,
                       warm_start=False)

In [61]:
grid_search.best_params_

{'max_features': 2, 'n_estimators': 30}

In [62]:
tuned_ = RandomForestClassifier(n_estimators=30 ,max_features=2)
tuned_.fit(trainx,trainy)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=2,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=30,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
tuned_train_preds = tuned_.predict(trainx)

In [0]:
tuned_test_preds = tuned_.predict(testx)

In [67]:
from sklearn.metrics import classification_report

report = classification_report(testy, tuned_test_preds)
print(report)

              precision    recall  f1-score   support

         0.0       0.81      1.00      0.89      1933
         1.0       0.00      0.00      0.00       467

    accuracy                           0.81      2400
   macro avg       0.40      0.50      0.45      2400
weighted avg       0.65      0.81      0.72      2400



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
